In [1]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath('../..'))

from offshore_wind_nj.config import INTERIM_DATA_DIR, ERA5_DATA_DIR
import matplotlib.pyplot as plt
from pyproj import Proj, Transformer

# import pygrib

import xarray as xr

2025-02-16 21:12:21.321 | INFO     | offshore_wind_nj.config:<module>:11 - PROJ_ROOT path is: /nfs/storage1/home/noriegac/Documents/Offshore_Wind_Research


In [2]:
import xarray as xr
print(xr.backends.plugins.list_engines())


{'netcdf4': <NetCDF4BackendEntrypoint>
  Open netCDF (.nc, .nc4 and .cdf) and most HDF5 files using netCDF4 in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.NetCDF4BackendEntrypoint.html, 'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'cfgrib': <CfGribBackend>
  Open GRIB files (.grib, .grib2, .grb and .grb2) in Xarray
  Learn more at https://github.com/ecmwf/cfgrib, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


/home/noriegac/.conda/envs/cris_env/lib/python3.12/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.26.0
  warnings.warn(


# Load the ERA5 Reanalysis dataset

In [4]:
era_path = os.path.join(ERA5_DATA_DIR, 'ERA5_2023_TO_2024.grib')

In [25]:
era5_ds = xr.open_dataset(era_path, engine='cfgrib')

/home/noriegac/.conda/envs/cris_env/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


## Inspect ERA5 Reanalysis dataset

In [33]:
era5_ds

<xarray.Dataset> Size: 1GB
Dimensions:     (time: 17544, latitude: 45, longitude: 65)
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 140kB 2023-01-01 ... 2024-12-31T23:00:00
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 360B 44.0 43.75 43.5 ... 33.5 33.25 33.0
  * longitude   (longitude) float64 520B -78.0 -77.75 -77.5 ... -62.25 -62.0
    valid_time  (time) datetime64[ns] 140kB ...
Data variables:
    u10         (time, latitude, longitude) float32 205MB ...
    v10         (time, latitude, longitude) float32 205MB ...
    u100        (time, latitude, longitude) float32 205MB ...
    v100        (time, latitude, longitude) float32 205MB ...
    u10n        (time, latitude, longitude) float32 205MB ...
    v10n        (time, latitude, longitude) float32 205MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-17T17:04 GRIB to CDM+CF via cfgrib-0.9.1...

### Check the dimensions

In [35]:
era5_ds.dims

FrozenMappingWarningOnValuesAccess({'time': 17544, 'latitude': 45, 'longitude': 65})

### Check coordinates

In [34]:
era5_ds.coords

Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 140kB 2023-01-01 ... 2024-12-31T23:00:00
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 360B 44.0 43.75 43.5 ... 33.5 33.25 33.0
  * longitude   (longitude) float64 520B -78.0 -77.75 -77.5 ... -62.25 -62.0
    valid_time  (time) datetime64[ns] 140kB ...

### Check data variables

In [36]:
era5_ds.variables

Frozen({'number': <xarray.Variable ()> Size: 8B
[1 values with dtype=int64]
Attributes:
    long_name:      ensemble member numerical id
    units:          1
    standard_name:  realization, 'time': <xarray.IndexVariable 'time' (time: 17544)> Size: 140kB
array(['2023-01-01T00:00:00.000000000', '2023-01-01T01:00:00.000000000',
       '2023-01-01T02:00:00.000000000', ..., '2024-12-31T21:00:00.000000000',
       '2024-12-31T22:00:00.000000000', '2024-12-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Attributes:
    long_name:      initial time of forecast
    standard_name:  forecast_reference_time, 'step': <xarray.Variable ()> Size: 8B
[1 values with dtype=timedelta64[ns]]
Attributes:
    long_name:      time since forecast_reference_time
    standard_name:  forecast_period, 'surface': <xarray.Variable ()> Size: 8B
[1 values with dtype=float64]
Attributes:
    long_name:  original GRIB coordinate for key: level(surface)
    units:      1, 'latitude': <xarray.IndexVariable 'latit

In [39]:
print(era5_ds.dims)
print(era5_ds.coords)
print(era5_ds.variables)


FrozenMappingWarningOnValuesAccess({'time': 17544, 'latitude': 45, 'longitude': 65})
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 140kB 2023-01-01 ... 2024-12-31T23:00:00
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 360B 44.0 43.75 43.5 ... 33.5 33.25 33.0
  * longitude   (longitude) float64 520B -78.0 -77.75 -77.5 ... -62.25 -62.0
    valid_time  (time) datetime64[ns] 140kB ...
Frozen({'number': <xarray.Variable ()> Size: 8B
[1 values with dtype=int64]
Attributes:
    long_name:      ensemble member numerical id
    units:          1
    standard_name:  realization, 'time': <xarray.IndexVariable 'time' (time: 17544)> Size: 140kB
array(['2023-01-01T00:00:00.000000000', '2023-01-01T01:00:00.000000000',
       '2023-01-01T02:00:00.000000000', ..., '2024-12-31T21:00:00.000000000',
       '2024-12-31T22:00:00.000000000', '2024-12-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Attributes:


### See available vars

In [38]:
era5_ds.data_vars  # See available variables.


Data variables:
    u10      (time, latitude, longitude) float32 205MB ...
    v10      (time, latitude, longitude) float32 205MB ...
    u100     (time, latitude, longitude) float32 205MB ...
    v100     (time, latitude, longitude) float32 205MB ...
    u10n     (time, latitude, longitude) float32 205MB ...
    v10n     (time, latitude, longitude) float32 205MB ...

# Extract Key variables

In [43]:
# era5_ds['v10']

In [44]:
# Extract key variables
u10 = era5_ds["u10"]  # 10m wind component U
v10 = era5_ds["v10"]  # 10m wind component V
time = era5_ds["time"]
lat = era5_ds["latitude"].values
lon = era5_ds["longitude"].values

# Load Sentinel-1 Data and Match Spatially

In [ ]:
# Example: Sentinel-1 data (assumed to be in DataFrame format)
sentinel_df = pd.read_csv("sentinel1_data.csv")  # Assuming it contains lat, lon, wind_speed
sentinel_coords = np.array(list(zip(sentinel_df["lat"], sentinel_df["lon"])))

In [46]:
# Build KDTree for fast nearest-neighbor search on GRIB data grid
grib_coords = np.array(np.meshgrid(lat, lon)).T.reshape(-1, 2)  # Flatten lat/lon grid
tree = cKDTree(grib_coords)

# Find closest GRIB grid points for each Sentinel-1 point
_, idxs = tree.query(sentinel_coords)

# Extract nearest lat/lon values from GRIB
matched_lat_lon = grib_coords[idxs]
sentinel_df["matched_lat"] = matched_lat_lon[:, 0]
sentinel_df["matched_lon"] = matched_lat_lon[:, 1]


45

# Match Temporally

In [ ]:
# Convert GRIB and Sentinel-1 times to pandas DateTime format
grib_times = pd.to_datetime(ds["time"].values)
sentinel_df["datetime"] = pd.to_datetime(sentinel_df["timestamp"])  # Assuming Sentinel timestamps

# Find closest GRIB timestamp for each Sentinel-1 record
sentinel_df["matched_time"] = sentinel_df["datetime"].apply(lambda x: grib_times[np.argmin(np.abs(grib_times - x))])


# Extract Wind Data

In [ ]:
# Function to extract wind data from GRIB dataset
def get_grib_wind(lat, lon, time):
    return ds.sel(latitude=lat, longitude=lon, time=time, method="nearest")

# Apply to Sentinel-1 matched points
sentinel_df["u10"] = sentinel_df.apply(lambda row: get_grib_wind(row.matched_lat, row.matched_lon, row.matched_time)["u10"].values, axis=1)
sentinel_df["v10"] = sentinel_df.apply(lambda row: get_grib_wind(row.matched_lat, row.matched_lon, row.matched_time)["v10"].values, axis=1)


# Reproject coordinate reference system


In [9]:
# Define the source and target CRS (WGS84 to LCC)
source_crs = "EPSG:4326"  # WGS84 (lat/lon)
target_crs = '+proj=lcc +lat_1=34.45660400390625 +lat_2=43.251747131347656 +lon_0=-71.37067794799805 +lat_0=38.85417556762695 +datum=WGS84'

# Create a Transformer object for converting from WGS84 to LCC
transformer = Transformer.from_crs(source_crs, target_crs)

# Convert lat/lon to projected x, y coordinates
# df['x'], df['y'] = transformer.transform(df['Lat'].values, df['Lon'].values)

In [8]:
# Check the coordinates (e.g., time, lat, lon)
print(ds.coords)

# Access specific coordinates, e.g., latitude and longitude
print(ds.coords['latitude'])
print(ds.coords['longitude'])


Coordinates:
    number         int64 8B ...
  * time           (time) datetime64[ns] 70kB 2023-01-01 ... 2023-12-31T23:00:00
    step           timedelta64[ns] 8B ...
    isobaricInhPa  float64 8B ...
  * latitude       (latitude) float64 312B 43.5 43.25 43.0 ... 34.5 34.25 34.0
  * longitude      (longitude) float64 344B -76.5 -76.25 -76.0 ... -66.25 -66.0
    valid_time     (time) datetime64[ns] 70kB ...
<xarray.DataArray 'latitude' (latitude: 39)> Size: 312B
array([43.5 , 43.25, 43.  , 42.75, 42.5 , 42.25, 42.  , 41.75, 41.5 , 41.25,
       41.  , 40.75, 40.5 , 40.25, 40.  , 39.75, 39.5 , 39.25, 39.  , 38.75,
       38.5 , 38.25, 38.  , 37.75, 37.5 , 37.25, 37.  , 36.75, 36.5 , 36.25,
       36.  , 35.75, 35.5 , 35.25, 35.  , 34.75, 34.5 , 34.25, 34.  ])
Coordinates:
    number         int64 8B ...
    step           timedelta64[ns] 8B ...
    isobaricInhPa  float64 8B ...
  * latitude       (latitude) float64 312B 43.5 43.25 43.0 ... 34.5 34.25 34.0
Attributes:
    units:         

Do I have to match Sentinel-1 products one by one?
Yes, since Sentinel-1 provides discrete SAR images with different timestamps and locations, you’ll need to process each product separately. The workflow would look like this:

Loop through Sentinel-1 products
Read the metadata (timestamp, lat/lon grid).
Find the closest GRIB timestamps and grid points.
Extract the corresponding wind data.
Store results
Save the matched data for each Sentinel-1 product.
Combine results into a larger dataset if needed.
If you're dealing with many Sentinel-1 products, consider automating the process with a script that loops through all available images.

2. How does the nearest-neighbor search work?
We use k-dimensional tree (KDTree) from scipy.spatial.cKDTree to efficiently find the nearest point in your GRIB grid. Here’s how it works:

Convert GRIB grid points into a KDTree

This organizes latitude-longitude points into a data structure optimized for fast searching.
It reduces the computational cost from O(n) (brute force) to O(log n) (KDTree search).
Query the nearest neighbor for each Sentinel-1 coordinate

For every Sentinel-1 lat/lon, the KDTree finds the closest GRIB grid point using Euclidean distance in 2D space.
💡 Alternative: Instead of a pure nearest-neighbor approach, you could use bilinear interpolation to compute values from the four nearest grid points.

3. Should I project coordinates before matching?
Using latitude and longitude directly assumes distances are uniform, which is not true, especially at high latitudes.

The Earth is not a perfect sphere; distances in degrees vary depending on location.
Using (lat, lon) as if they were Cartesian coordinates can introduce small errors.
Best Approach: Use an Equal-Area Projection (like Lambert)
Why?

It preserves distances locally, making nearest-neighbor searches more accurate.
Wind data typically benefits from Lambert Conformal Conic (LCC) projection, commonly used for meteorological models.
How to Reproject to Lambert Projection (EPSG:4326 → EPSG:3031 or LCC)?
You can transform the coordinates using pyproj:

python
Copy
Edit
from pyproj import Transformer

# Define transformation from WGS84 (lat/lon) to Lambert projection
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3031", always_xy=True)

# Convert GRIB coordinates
grib_x, grib_y = transformer.transform(lon, lat)

# Convert Sentinel-1 coordinates
sentinel_x, sentinel_y = transformer.transform(sentinel_df["lon"], sentinel_df["lat"])
Now, use (grib_x, grib_y) instead of (lat, lon) in the KDTree nearest-neighbor search.

What If You Skip Projection?
If your study area is small (e.g., offshore wind near New Jersey), errors may be minor.
If working globally, projection is highly recommended.
Final Thoughts
If Sentinel-1 images are spread over time and space, loop through them product by product.
Using KDTree nearest-neighbor is fast, but consider bilinear interpolation for smoother values.
Projecting to Lambert (or another suitable projection) is best, especially for accurate spatial matching.
Would you like help implementing the Lambert projection step in your workflow? 🚀